# 모든 model에 대해 각각 model training + 5-fold + SHAP -> 비교
- catBoost, XGBoost, Randomforest, Deepforest 총 4가지 모델 사용

- feature들 간의 상관계수를 계산하는 방법은 오직 선형성만 파악하고 각 인자의 중요도는 파악하기 어렵기 때문에 SHAP를 사용함
- 모델마다 각 인자의 중요도를 해석하는 기준과 방식이 다르고 동일한 인자라도 모델에 따라 영향력을 다르게 판단하기 때문에 각 모델마다 중요도 분석을 진행함
ex) Catboost : 범주형 인자를 잘 처리하므로 이 인자의 중요도가 높게 나올 수 有
    XGBoost : tree split 기준에서 빠르게 좋은 gain을 얻는 인자에 편향될 수 有
    Randomforest : 다양한 tree에 균등하게 기여한 인자에 중요도를 부여함


In [ ]:
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from deepforest import CascadeForestRegressor


In [ ]:
df = pd.read_csv("dataset.csv")
X = df[['cement', 'slag', 'fly_ash', 'water', 'superplasticizer', 'coarse_aggregate', 'fine_aggregate', 'age']]
y = df['CCS']

In [ ]:
# 사용할 모델 정의
models = {
    "CatBoost": CatBoostRegressor(verbose=0, random_state=42),
    "XGBoost": XGBRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(n_estimators=200, random_state=42),
    "DeepForest": CascadeForestRegressor(random_state=42)
}


In [ ]:
# 5-fold validation 기본 설정
kf = KFold(n_splits=5, shuffle=True, random_state=42)
feature_names = X.columns
results = {}

In [ ]:
# 각 모델마다 validation + SHAP 계산
for model_name, model in models.items():
    print(f"Processing {model_name}...")
    shap_values_all = []

    for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)

        # SHAP Explainer
        explainer = shap.Explainer(model, X_test)
        shap_values = explainer(X_test)
        shap_values_all.append(abs(shap_values.values)) 

    # fold 평균 SHAP → feature importance
    mean_shap = np.mean(np.array(shap_values_all), axis=0)
    feature_importance = pd.Series(mean_shap.mean(axis=0), index=feature_names)
    results[model_name] = feature_importance

In [ ]:
# 결과 table로 나타내기
importance_df = pd.DataFrame(results)
importance_df = importance_df.loc[importance_df.mean(axis=1).sort_values(ascending=False).index]

print("\nFeature Importance Comparison (Mean SHAP Values across 5 folds):")
print(importance_df)


In [ ]:
# 시각화를 위해 마지막 fold 기준 summary plot 그려보기
for model_name, model in models.items():
    print(f"\n📈 SHAP Summary Plot for {model_name}")
    model.fit(X, y)
    explainer = shap.Explainer(model, X)
    shap_values = explainer(X)
    shap.summary_plot(shap_values, X)

모든 fold의 SHAP summary plot을 그리는 것은 많은 연산이 필요하고 코드도 꽤나 복잡하기 때문에 그 중 하나만 대표적으로 보여주는 의미로 마지막  fold 기준을 사용함
-> test set이 전체 데이터를 잘 반영한다면 하나의 fold를 보더라도 중요한 인자의 경향은 비슷하게 나오고 전체적인 중요 feature 순위나 분포를 대부분 유사하므로 하나만 사용해도 괜찮을 것 같다고 판단